In [ ]:
%reload_ext autoreload
%autoreload 2
import pandas as pd
from acr.nor import *
import matplotlib.pyplot as plt
import seaborn as sns
import acr
from acr.nor import stim_subjects
import numpy as np
import pingouin as pg
from acr.plots import lrg, pub

#--------------------------------- Import Publication Functions ---------------------------------#
pub_utils = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pub_utils')
from pub_utils import write_stats, write_source_data

In [ ]:
dfs = []
all_subjects = get_all_subjects()
for sub in all_subjects:
    print(sub)
    t = get_subject_type(sub)
    dfs.append(load_subject_node(sub, 'nose', test_dur=6000, type=t))
df = pd.concat(dfs)

# OVERVIEW

In [ ]:
plt.rcdefaults()
plt.style.use('fivethirtyeight')

In [ ]:

frame_counts = df.groupby(['subject', 'type', 'condition', 'node', 'side']).count().reset_index()
counts = frame_counts.sort_values(by=['subject', 'side'])
counts['ratio'] = counts.groupby(['subject', 'condition'])['x'].transform(lambda x: x / x.sum())

In [ ]:
# For the raw time-on-side comparisons, we obviously need to exclude the subjects who did not spend a fair amount of time on both sides during the acquisition period
to_ex = counts.loc[(counts['condition']=='acq')&(counts['ratio']>(2/3))]['subject'].values
df_new = df.loc[~df['subject'].isin(to_ex)]
frame_counts = df_new.groupby(['subject', 'type', 'condition', 'node', 'side']).count().reset_index()
counts = frame_counts.sort_values(by=['subject', 'side'])
counts['ratio'] = counts.groupby(['subject', 'condition'])['x'].transform(lambda x: x / x.sum())

In [ ]:
f, axs = plt.subplots(3, 1, figsize=(30, 10))
axs[0] = sns.barplot(counts.loc[(counts['side']=='novel')&(counts['type']=='sleep')], x='subject', y='ratio', hue='condition', palette=['gray', 'green'], ax=axs[0])
axs[1] = sns.barplot(counts.loc[(counts['side']=='novel')&(counts['type']=='sd')], x='subject', y='ratio', hue='condition', palette=['gray', 'green'], ax=axs[1])
axs[2] = sns.barplot(counts.loc[(counts['side']=='novel')&(counts['type']=='stim')], x='subject', y='ratio', hue='condition', palette=['gray', 'green'], ax=axs[2])
for a in axs:
    a.axhline(0.5, color='red', linestyle='--')

In [ ]:
f, axs = plt.subplots(1, 1, figsize=(30, 20))
axs = sns.barplot(counts.loc[(counts['side']=='novel')&(counts['type']=='stim')], x='subject', y='ratio', hue='condition', palette=['gray', 'green'], ax=axs)
axs.axhline(0.5, color='red', linestyle='--')

In [ ]:
g = sns.catplot(counts.loc[counts['side']=='novel'], kind='bar', x='condition', y='ratio', hue='condition', palette=['gray', 'green'], col='type', col_order=['sleep', 'sd', 'stim'])
for a in g.axes.flat:
    a.axhline(0.5, color='red', linestyle='--')

# Basic Group-level plots

In [ ]:
cs = counts.loc[counts['side']=='novel']
cs_sleep = cs.loc[cs['type']=='sleep']
cs_sd = cs.loc[cs['type']=='sd']
cs_stim = cs.loc[cs['type']=='stim']
cs_sleep_acq = cs_sleep.loc[cs_sleep['condition']=='acq']['ratio'].values
cs_sleep_test = cs_sleep.loc[cs_sleep['condition']=='test']['ratio'].values
cs_sd_acq = cs_sd.loc[cs_sd['condition']=='acq']['ratio'].values
cs_sd_test = cs_sd.loc[cs_sd['condition']=='test']['ratio'].values
cs_stim_acq = cs_stim.loc[cs_stim['condition']=='acq']['ratio'].values
cs_stim_test = cs_stim.loc[cs_stim['condition']=='test']['ratio'].values

In [ ]:
rel_sleep = cs_sleep_test/cs_sleep_acq
rel_sd = cs_sd_test/cs_sd_acq
rel_stim = cs_stim_test/cs_stim_acq

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.boxplot([cs_sleep_acq, cs_sleep_test], positions=[0.3, 0.7])
for i in range(len(cs_sleep_acq)):
    ax.scatter(0.4, cs_sleep_acq[i], color='gray', alpha=0.9, s=60, zorder=202)
    ax.scatter(0.6, cs_sleep_test[i], color='green', alpha=0.9, s=60, zorder=203)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.boxplot([cs_sd_acq, cs_sd_test], positions=[0.3, 0.7])
for i in range(len(cs_sd_acq)):
    ax.scatter(0.4, cs_sd_acq[i], color='gray', alpha=0.9, s=60, zorder=202)
    ax.scatter(0.6, cs_sd_test[i], color='green', alpha=0.9, s=60, zorder=203)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(10, 5))
ax.boxplot([cs_stim_acq, cs_stim_test], positions=[0.3, 0.7])
for i in range(len(cs_stim_acq)):
    ax.scatter(0.4, cs_stim_acq[i], color='gray', alpha=0.9, s=60, zorder=202)
    ax.scatter(0.6, cs_stim_test[i], color='green', alpha=0.9, s=60, zorder=203)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(20, 10))
ax.boxplot([cs_sleep_test, cs_sd_test, cs_stim_test], positions=[0.3, 0.5, 0.7], showfliers=False, showmeans=True)
for i in range(len(cs_sleep_test)):
    ax.scatter(0.3, cs_sleep_test[i], color='red', alpha=0.9, s=60, zorder=202)
for i in range(len(cs_sd_test)):
    ax.scatter(0.5, cs_sd_test[i], color='gray', alpha=0.9, s=60, zorder=203)
for i in range(len(cs_stim_test)):
    ax.scatter(0.7, cs_stim_test[i], color='blue', alpha=0.9, s=60, zorder=204)
ax.axhline(0.5, color='red', linestyle='--')
ax.set_title("RAW NOVELTY RATIO DURING TEST")

# Write Stats and Source Data

In [ ]:
subs_to_exclude = []
csex = cs.copy()
for sub in subs_to_exclude:
    csex = csex.loc[csex['subject']!=sub]
cs_sleep = csex.loc[cs['type']=='sleep']
cs_sd = csex.loc[cs['type']=='sd']
cs_stim = csex.loc[cs['type']=='stim']
cs_sleep_acq = cs_sleep.loc[cs_sleep['condition']=='acq']['ratio'].values
cs_sleep_test = cs_sleep.loc[cs_sleep['condition']=='test']['ratio'].values
cs_sd_acq = cs_sd.loc[cs_sd['condition']=='acq']['ratio'].values
cs_sd_test = cs_sd.loc[cs_sd['condition']=='test']['ratio'].values
cs_stim_acq = cs_stim.loc[cs_stim['condition']=='acq']['ratio'].values
cs_stim_test = cs_stim.loc[cs_stim['condition']=='test']['ratio'].values

In [ ]:
# Normality Checks
from scipy.stats import shapiro
_, p = shapiro(cs_sleep_test)
print(f'p-value for sleep: {p}')

_, p = shapiro(cs_stim_test)
print(f'p-value for stim: {p}')

_, p = shapiro(cs_sd_test)
print(f'p-value for sd: {p}')

In [ ]:
sleep_df = pd.DataFrame({"sleep_condition":'sleep', 'ratio':cs_sleep_test})
stim_df = pd.DataFrame({"sleep_condition":'stim', 'ratio':cs_stim_test})
sd_df = pd.DataFrame({"sleep_condition":'sd', 'ratio': cs_sd_test})

anova_df = pd.concat([sleep_df, stim_df, sd_df])

In [ ]:
# Equal Variance Check
pg.homoscedasticity(data=anova_df, dv='ratio', group='sleep_condition')

In [ ]:
anov = pg.welch_anova(data=anova_df, dv='ratio', between='sleep_condition')
anov

In [ ]:
tuk = pg.pairwise_tukey(anova_df, dv='ratio', between='sleep_condition')
tuk

In [ ]:
import scipy
trts = [cs_sleep_test, cs_stim_test]
DUN = scipy.stats.dunnett(*trts, control=cs_sd_test)

In [ ]:
acr.stats.write_stats_result('novelty_ratio_anova', 'WELCH_ANOVA', anov['F'][0], anov['p-unc'][0], 'np2', anov['np2'][0])
acr.stats.write_stats_result('sleep_vs_sd_novelty_ratio', 'pairwise_tukey', tuk['T'][0], tuk['p-tukey'][0], 'g', tuk['hedges'][0])
acr.stats.write_stats_result('stim_vs_sd_novelty_ratio', 'pairwise_tukey', tuk['T'][1], tuk['p-tukey'][1], 'g', tuk['hedges'][1])
acr.stats.write_stats_result('sleep_vs_stim_novelty_ratio', 'pairwise_tukey', tuk['T'][2], tuk['p-tukey'][2], 'g', tuk['hedges'][2])

In [ ]:
# write source data for statistical tests
source_data_sleep = pd.DataFrame({'novelty_ratio': cs_sleep_test, 'subject_number': range(len(cs_sleep_test)), 'subject_type': 'sleep'})
source_data_sd = pd.DataFrame({'novelty_ratio': cs_sd_test, 'subject_number': range(len(cs_sd_test)), 'subject_type': 'sd'})
source_data_stim = pd.DataFrame({'novelty_ratio': cs_stim_test, 'subject_number': range(len(cs_stim_test)), 'subject_type': 'sd-off-induction'})
source_data = pd.concat([source_data_sleep, source_data_sd, source_data_stim])
write_source_data(source_data, 'floor_texture_recognition_source_data')

# Final Plot

In [ ]:
plt.rcdefaults()
lrg()
plt.rcParams['xtick.bottom'] = False
from acr.utils import SOM_BLUE, NNXR_GRAY, NREM_RED, PAPER_FIGURE_ROOT

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(9, 5))
# Boxplots for all three conditions

ax, box = acr.plots.add_boxplot(ax, cs_sleep_test, positions=[0.4], widths=0.0225, color=NREM_RED, means=True, mean_color='gold', mean_linewidth=3.5, mean_linestyle='--', mean_dashes=(1.2, 1.2), alpha=0.9)
ax = acr.plots.add_data_points(ax, cs_sleep_test, x_pos=0.42, color=NREM_RED, alpha=0.9, s=70, zorder=202)

ax, box2 = acr.plots.add_boxplot(ax, cs_sd_test, positions=[0.5], widths=0.0225, color=NNXR_GRAY, means=True, mean_color='gold', mean_linewidth=3.5, mean_linestyle='--', mean_dashes=(1.2, 1.2), alpha=0.9)
ax = acr.plots.add_data_points(ax, cs_sd_test, x_pos=0.52, color=NNXR_GRAY, alpha=0.9, s=70, zorder=203)

ax, box3 = acr.plots.add_boxplot(ax, cs_stim_test, positions=[0.6], widths=0.0225, color=SOM_BLUE, means=True, mean_color='gold', mean_linewidth=3.5, mean_linestyle='--', mean_dashes=(1.2, 1.2), alpha=0.9)
ax = acr.plots.add_data_points(ax, cs_stim_test, x_pos=0.62, color=SOM_BLUE, alpha=0.9, s=70, zorder=204)

ax.set_xlim(0.35, 0.65)
ax.set_ylim(0.3, 0.8)
ax.set_yticks([0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
#ax.axhline(0.5, color='red', linestyle='--')

plt.tight_layout()
f.savefig(f'{PAPER_FIGURE_ROOT}/nor_behavior/novelty_ratio_boxplot.png', dpi=600, bbox_inches='tight', transparent=True)

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(9, 5))
# Boxplots for all three conditions
box_sleep = ax.boxplot(cs_sleep_test, positions=[0.4], widths=0.025, patch_artist=True, capprops=dict(color='none', linewidth=0), whiskerprops=dict(color='k', linewidth=3), medianprops=dict(color='k', linewidth=3, zorder=101), showfliers=False)
box_sleep['boxes'][0].set_facecolor(NREM_RED)
box_sleep['boxes'][0].set_alpha(0.8)
box_sleep['boxes'][0].set_linewidth(0)
box_sleep['boxes'][0].set_zorder(100)

box_sd = ax.boxplot(cs_sd_test, positions=[0.5], widths=0.025, patch_artist=True, capprops=dict(color='none', linewidth=0), whiskerprops=dict(color='k', linewidth=3), medianprops=dict(color='k', linewidth=3, zorder=101), showfliers=False)
box_sd['boxes'][0].set_facecolor(NNXR_GRAY)
box_sd['boxes'][0].set_alpha(0.8)
box_sd['boxes'][0].set_linewidth(0)
box_sd['boxes'][0].set_zorder(100)

box_stim = ax.boxplot(cs_stim_test, positions=[0.6], widths=0.025, patch_artist=True, capprops=dict(color='none', linewidth=0), whiskerprops=dict(color='k', linewidth=3), medianprops=dict(color='k', linewidth=3, zorder=101), showfliers=False)
box_stim['boxes'][0].set_facecolor(SOM_BLUE)
box_stim['boxes'][0].set_alpha(0.8)
box_stim['boxes'][0].set_linewidth(0)
box_stim['boxes'][0].set_zorder(100)

# Add individual points
for i in range(len(cs_sleep_test)):
    ax.scatter(0.42, cs_sleep_test[i], color=NREM_RED, alpha=0.9, s=70, zorder=202)
for i in range(len(cs_sd_test)):
    ax.scatter(0.52, cs_sd_test[i], color=NNXR_GRAY, alpha=0.9, s=70, zorder=203)
for i in range(len(cs_stim_test)):
    ax.scatter(0.62, cs_stim_test[i], color=SOM_BLUE, alpha=0.9, s=70, zorder=204)

ax.set_xlim(0.35, 0.65)
ax.set_ylim(0.3, 0.8)
ax.set_yticks([0.3, 0.4, 0.5, 0.6, 0.7, 0.8])
ax.axhline(0.5, color='red', linestyle='--')
plt.tight_layout()
f.savefig(f'{PAPER_FIGURE_ROOT}/nor_behavior/novelty_ratio_boxplot.png', dpi=600, bbox_inches='tight', transparent=True)

# Relative Data (each subject normalized to its own acquisition)

In [ ]:
frame_counts = df.groupby(['subject', 'type', 'condition', 'node', 'side']).count().reset_index()
counts = frame_counts.sort_values(by=['subject', 'side'])
counts['ratio'] = counts.groupby(['subject', 'condition'])['x'].transform(lambda x: x / x.sum())

In [ ]:
cs = counts.loc[counts['side']=='novel']
cs_sleep = cs.loc[cs['type']=='sleep']
cs_sd = cs.loc[cs['type']=='sd']
cs_stim = cs.loc[cs['type']=='stim']
cs_sleep_acq = cs_sleep.loc[cs_sleep['condition']=='acq']['ratio'].values
cs_sleep_test = cs_sleep.loc[cs_sleep['condition']=='test']['ratio'].values
cs_sd_acq = cs_sd.loc[cs_sd['condition']=='acq']['ratio'].values
cs_sd_test = cs_sd.loc[cs_sd['condition']=='test']['ratio'].values
cs_stim_acq = cs_stim.loc[cs_stim['condition']=='acq']['ratio'].values
cs_stim_test = cs_stim.loc[cs_stim['condition']=='test']['ratio'].values

In [ ]:
rel_sleep = cs_sleep_test/cs_sleep_acq
rel_sd = cs_sd_test/cs_sd_acq
rel_stim = cs_stim_test/cs_stim_acq

In [ ]:
plt.rcdefaults()
acr.plots.supl()

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(20, 10))
ax.boxplot([rel_sleep, rel_sd, rel_stim], positions=[0.3, 0.5, 0.7], showfliers=False, showmeans=True)
for i in range(len(rel_sleep)):
    ax.scatter(0.3, rel_sleep[i], color='red', alpha=0.9, s=60, zorder=202)
for i in range(len(rel_sd)):
    ax.scatter(0.5, rel_sd[i], color='gray', alpha=0.9, s=60, zorder=203)
for i in range(len(rel_stim)):
    ax.scatter(0.7, rel_stim[i], color='blue', alpha=0.9, s=60, zorder=204)
ax.axhline(1, color='red', linestyle='--')
ax.set_title("RELATIVE NOVELTY RATIO DURING TEST (per subject adjusted)")

In [ ]:
sleep_df = pd.DataFrame({"sleep_condition":'sleep', 'ratio':rel_sleep})
stim_df = pd.DataFrame({"sleep_condition":'stim', 'ratio':rel_stim})
sd_df = pd.DataFrame({"sleep_condition":'sd', 'ratio': rel_sd})

anova_df = pd.concat([sleep_df, stim_df, sd_df])

In [ ]:
# Equal Variance Check
pg.homoscedasticity(data=anova_df, dv='ratio', group='sleep_condition')

In [ ]:
anov = pg.welch_anova(data=anova_df, dv='ratio', between='sleep_condition')
anov

In [ ]:
scipy.stats.dunnett(*[rel_sleep, rel_stim], control=rel_sd)


In [ ]:
tuk = pg.pairwise_tukey(anova_df, dv='ratio', between='sleep_condition')
tuk

In [ ]:
tt_rel_sleep = pg.ttest(rel_sleep, rel_sd)
tt_rel_sleep

In [ ]:
tt_rel_stim = pg.ttest(rel_stim, rel_sd)
tt_rel_stim